# Pandas Mixed Type Difficulties

This notebook demonstrates some challenges that arise when you are doing a lot of data cleaning with pandas. Read through to learn what problems may occur and how to avoid them!

## How Mixed Types Occur and the Object data type 

In [3]:
import numpy as np
import pandas as pd

Let's say that you have a couple of files, some sales data in one file and some census data in another file. You want to merge these two datasets on zip code.

In [4]:
sales_data = pd.read_csv("data/sales_data.csv",
                         header=0,
                         parse_dates=["date"])

In [5]:
sales_data.shape

(594, 6)

In [6]:
sales_data.sample(3, random_state=1)

,date,units_sold,address_line,city,state,zip_code
559,2018-06-18 17:14:37,0,615 E 89TH,CA,USA,90002-1635
347,2019-04-12 19:27:48,8,8800 WADSWORTH,CA,USA,NaN
396,2019-07-24 07:45:57,2,9914 S BROADWAY,CA,USA,90003-4173


Looks like there are some missing zip codes

In [7]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Data columns (total 6 columns):
date            594 non-null datetime64[ns]
units_sold      594 non-null int64
address_line    594 non-null object
city            594 non-null object
state           594 non-null object
zip_code        521 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 27.9+ KB


In [8]:
census_data = pd.read_csv("us-population-by-zip-code/population_by_zip_2010.csv",
                          header=0)

In [9]:
census_data.head(3)

,population,minimum_age,maximum_age,gender,zipcode,geo_id
0,50,30.0,34.0,female,61747,8600000US61747
1,5,85.0,NaN,male,64120,8600000US64120
2,1389,30.0,34.0,male,95117,8600000US95117


In [10]:
population = census_data.groupby("zipcode")["population"].sum().reset_index()

We want to merge these two dataframe on zip-code, but the sales data has a 9 digit zip code and the census data has 5 digit zip codes. Let's fix this.

In [11]:
sales_data["zip_code"] = sales_data["zip_code"].str[:5]

Now let's also fix the missing zip codes. Let's just say we know that all the zip codes before 2018 were 90001 and all the missing zip codes after that were 90002.

In [12]:
sales_data.loc[sales_data["zip_code"].isna() & (sales_data["date"] < pd.to_datetime("2018-01-01")), "zip_code"] = 90001
sales_data.loc[sales_data["zip_code"].isna() & (sales_data["date"] >= pd.to_datetime("2018-01-01")), "zip_code"] = 90002

In [13]:
sales_data["zip_code"].isna().sum()

0

In [14]:
sales_and_population = pd.merge(sales_data, population, left_on="zip_code", right_on="zipcode", validate="m:1")

In [15]:
sales_and_population.head()

,date,units_sold,address_line,city,state,zip_code,zipcode,population
0,2019-01-10 10:34:58,15,1411 E 61ST,CA,USA,90002,90002,153669
1,2018-01-03 01:58:35,17,200 W 91ST,CA,USA,90002,90002,153669
2,2019-05-09 18:20:39,-1,407 N WILTON,CA,USA,90002,90002,153669
3,2018-04-20 07:57:47,18,7200 AVALON,CA,USA,90002,90002,153669
4,2018-07-03 22:56:37,8,501 N ROSSMORE,CA,USA,90002,90002,153669


Seems okay, right?

In [16]:
sales_and_population.shape

(73, 8)

Wait, that doesn't seem right. Why are we missing so many rows?!

Turns out it is because we had replaced the missing zip codes with an integer, which correctly joined to the integer type in the population dataframe. The remaining zip code data that is actually a string did not join at all with this data, and now we are missing it!



In [17]:
sales_data.dtypes

date            datetime64[ns]
units_sold               int64
address_line            object
city                    object
state                   object
zip_code                object
dtype: object

In [18]:
population.dtypes

zipcode       int64
population    int64
dtype: object

In [19]:
print(set(sales_data['zip_code'].apply(lambda x: type(x))))

{<class 'str'>, <class 'int'>}


So the takeaway is the 'object' column can contain mixed types. Underlying it is a numpy Python object dtype, which is just pointers to Python objects. This is used for data like strings because numpy has a fixed storage allocation for strings, which is too limiting to be used for most data, so to allow for more variable memory in a numpy array, we must use the object dtype. But since floats and ints are also Python objects, it allows for this mixed typing. See more in References below.

It will have to be an object (at least until Pandas 1.0) for all strings, but be wary when you are converting from strings to data that can also be interpreted in another format (like dates, ints, floats). 

You need to also be careful when subsetting out data that has mixed types as shown below:

In [20]:
print(sales_data.loc[sales_data["zip_code"] == '90001'].shape[0])
print(sales_data.loc[sales_data["zip_code"] == 90001].shape[0])

129
25


Now, if you case it to a string, then you're not going to have the same dtypes on the left and right dataframe's join column. So you'll receive a null intersection.

In [21]:
sales_data = sales_data.astype({"zip_code": str})

In [22]:
sales_and_population = pd.merge(sales_data, population, left_on="zip_code", right_on="zipcode", validate="m:1")

In [23]:
len(sales_and_population)

0

We must make sure we use the same dtype.

In [24]:
sales_data = sales_data.astype({"zip_code": int})
sales_and_population = pd.merge(sales_data, population, left_on="zip_code", right_on="zipcode", validate="m:1")
print(len(sales_and_population))

594


# Example of creating a mixed type

In [25]:
df = pd.DataFrame({"price": [9.99, "$9.99"]})

In [26]:
df.dtypes

price    object
dtype: object

In [28]:
list(map(type, df["price"]))

[float, str]

# References
- [Census data](https://www.kaggle.com/census/us-population-by-zip-code)
- [Source of random addresses](https://ca.postcodebase.com/)
- [Another example showing similar issue when working with currencies](https://pbpython.com/currency-cleanup.html)
- [Explanation of NumPy data types](https://stackoverflow.com/questions/34881079/pandas-distinction-between-str-and-object-types)
